# Instalación librería

# Carga librerías

In [4]:
from keras.models import Sequential
from keras.layers import (Conv2D, BatchNormalization, ELU, AveragePooling2D,
                          Dropout, Permute, Reshape, GRU, Dense, DepthwiseConv2D,
                          Activation, SeparableConv2D)
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from keras.regularizers import l2
import tensorflow as tf

import numpy as np
import pandas as pd

from SignalProcessor.Filter import Filter
from SignalProcessor.CSPMulticlass import CSPMulticlass
from SignalProcessor.FeatureExtractor import FeatureExtractor
from SignalProcessor.RavelTransformer import RavelTransformer


## Clasificadores LDA y SVM
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC

from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, classification_report, precision_recall_fscore_support
import pickle

from sklearn.metrics import roc_auc_score, roc_curve, auc
from sklearn.preprocessing import label_binarize

from keras.optimizers import Adam

from keras.utils import to_categorical

from TrialsHandler.TrialsHandler import TrialsHandler
from TrialsHandler.Concatenate import Concatenate

# Subida de datos

In [5]:
np.random.seed(42)
tf.random.set_seed(42)

### ********** Cargamos los datos **********
file = "data/sujeto_1/eegdata/sesion1/sn1_ts0_ct0_r1.npy"
eventosFile = "data/sujeto_1/eegdata/sesion1/sn1_ts0_ct0_r1_events.txt"
rawEEG_1 = np.load(file)
eventos_1 = pd.read_csv(eventosFile, sep = ",")

file = "data/sujeto_1/eegdata/sesion2/sn2_ts0_ct0_r1.npy"
eventosFile = "data/sujeto_1/eegdata/sesion2/sn2_ts0_ct0_r1_events.txt"
rawEEG_2 = np.load(file)
eventos_2 = pd.read_csv(eventosFile, sep = ",")

#Creamos objetos para manejar los trials
th_1 = TrialsHandler(rawEEG_1, eventos_1, tinit = 0, tmax = 4, reject=None, sample_rate=250.)
th_2 = TrialsHandler(rawEEG_2, eventos_2, tinit = 0, tmax = 4, reject=None, sample_rate=250.)

dataConcatenada = Concatenate([th_1, th_2])#concatenamos datos

channelsSelected = [0,1,2,3,6,7]

#me quedo con channelsSelected
dataConcatenada.trials = dataConcatenada.trials[:,channelsSelected,:]

# Estas son las clases que quieres mantener
        # "Mano Izquierda",1
        # "Mano Derecha",2
        # "Ambas Manos",3
        # "Pies",4
        # "Rest"5

desired_classes = [1, 2]

num_classes = len(desired_classes)

# Filtramos los ensayos y etiquetas para mantener solo las clases deseadas
filtered_indices = np.isin(dataConcatenada.labels, desired_classes)
trials = dataConcatenada.trials[filtered_indices]
labels = dataConcatenada.labels[filtered_indices]

if (5 in desired_classes) and (4 in desired_classes):
  labels[labels == 4] = 3
  labels[labels == 5] = 4
elif (5 in desired_classes):
  labels[labels == 5] = 3

print(trials.shape)
print(labels.shape)

labels = labels-1

eeg_train, eeg_test, labels_train, labels_test = train_test_split(trials, labels, test_size=0.2, stratify=labels)

### ********** Instanciamos los diferentes objetos que usaremos en el pipeline**********

fm = 250. #frecuencia de muestreo
filtro = Filter(highcut = 30)
csp = CSPMulticlass(n_components=2, method = "ovo", n_classes = num_classes, reg = 0.01)
featureExtractor = FeatureExtractor(method = "welch", sample_rate = fm, axisToCompute=2, band_values=[8,12])
ravelTransformer = RavelTransformer()

Se han extraido 75 trials
Se han extraido 8 canales
Se han extraido 1000 muestras por trial
Se han extraido 75 trials
Se han extraido 8 canales
Se han extraido 1000 muestras por trial
(60, 6, 1000)
(60,)


# LDA

In [6]:
# Instanciamos un LDA
lda = LinearDiscriminantAnalysis()  # instanciamos el clasificador LDA

### ********** Creamos el pipeline para LDA **********

pipeline_lda = Pipeline([
    ('pasabanda', filtro),
    ('cspmulticlase', csp),
    ('featureExtractor', featureExtractor),
    ('ravelTransformer', ravelTransformer),
    ('lda', lda)
])

### ********** Creamos la grilla de hiperparámetros **********

param_distributions_lda = {
    'pasabanda__lowcut': [5],
    'pasabanda__highcut': [16],
    'cspmulticlase__n_components': [2, 3],
    'cspmulticlase__method': ["ovo","ova"],
    'cspmulticlase__reg': [0.01],
    'cspmulticlase__log': [None],
    'cspmulticlase__norm_trace': [False],
    'featureExtractor__method': ["welch", "hilbert"],
    'featureExtractor__sample_rate': [fm],
    'featureExtractor__band_values': [[8, 12]],
    'lda__solver': ['svd', 'lsqr'],
    'lda__n_components': [None, 1], # Ajusta según tu número de clases y características.
    'lda__store_covariance': [True, False],
    'lda__tol': [0.001, 0.01]
}

# Creamos el RandomizedSearch para el LDA
# Aquí puedes ajustar n_iter según cuántas iteraciones de búsqueda aleatoria quieras realizar
random_search_lda = RandomizedSearchCV(pipeline_lda, param_distributions=param_distributions_lda, n_iter=10, cv=5, n_jobs=1, verbose=0, random_state=42)

### ********** Entrenamos el modelo **********
random_search_lda.fit(eeg_train, labels_train)
### ******************************************

### Nos quedamos con el mejor estimador
best_lda = random_search_lda.best_estimator_

# Reporte de clasificación
y_true, y_pred = labels_test, best_lda.predict(eeg_test)
print(classification_report(y_true, y_pred), end="\n\n")

## Creamos una matriz de confusión
cm_lda = confusion_matrix(y_true, y_pred)
print(cm_lda)

## obtenemos precision, recall y f1-score y los guardamos en variables
precision_lda, recall_lda, f1score_lda, _ = precision_recall_fscore_support(y_true, y_pred, average='macro')

## Obtenemos el accuracy y lo redondeamos a 2 decimales
acc_lda = accuracy_score(y_true, y_pred)
acc_lda = np.round(acc_lda, decimals=2) * 100
print(f"El accuracy del mejor clasificador LDA es de {acc_lda}")

Computing rank from data with rank=None
    Using tolerance 1.1 (2.2e-16 eps * 6 dim * 8.4e+14  max singular value)
    Estimated rank (mag): 6
    MAG: rank 6 computed from 6 data channels with 0 projectors
Reducing data rank from 6 -> 6
Estimating covariance using SHRINKAGE
Done.
Computing rank from data with rank=None
    Using tolerance 1.1 (2.2e-16 eps * 6 dim * 7.9e+14  max singular value)
    Estimated rank (mag): 6
    MAG: rank 6 computed from 6 data channels with 0 projectors
Reducing data rank from 6 -> 6
Estimating covariance using SHRINKAGE
Done.


Computing rank from data with rank=None
    Using tolerance 1.1 (2.2e-16 eps * 6 dim * 8.5e+14  max singular value)
    Estimated rank (mag): 6
    MAG: rank 6 computed from 6 data channels with 0 projectors
Reducing data rank from 6 -> 6
Estimating covariance using SHRINKAGE
Done.
Computing rank from data with rank=None
    Using tolerance 1.1 (2.2e-16 eps * 6 dim * 8.1e+14  max singular value)
    Estimated rank (mag): 6
    MAG: rank 6 computed from 6 data channels with 0 projectors
Reducing data rank from 6 -> 6
Estimating covariance using SHRINKAGE
Done.
Computing rank from data with rank=None
    Using tolerance 1.1 (2.2e-16 eps * 6 dim * 8.5e+14  max singular value)
    Estimated rank (mag): 6
    MAG: rank 6 computed from 6 data channels with 0 projectors
Reducing data rank from 6 -> 6
Estimating covariance using SHRINKAGE
Done.
Computing rank from data with rank=None
    Using tolerance 1.1 (2.2e-16 eps * 6 dim * 8e+14  max singular value)
    Estimated rank (mag): 6
    MAG: 

c:\Users\Admin\miniconda3\envs\bcihack-GUI\Lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
20 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\Admin\miniconda3\envs\bcihack-GUI\Lib\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\Admin\miniconda3\envs\bcihack-GUI\Lib\site-packages\sklearn\base.py", line 1151, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Admin\miniconda3\envs\bcihack-GUI\Lib\site-packa

In [7]:
if num_classes == 2:
  from sklearn.metrics import roc_curve, auc, classification_report

  # Reporte de clasificación
  y_true, y_pred = labels_test, best_lda.predict(eeg_test)
  print(classification_report(y_true, y_pred), end="\n\n")

  # Obtener la función de decisión o probabilidad para las clases
  try:
      y_score = best_lda.decision_function(eeg_test)
  except AttributeError:
      y_prob = best_lda.predict_proba(eeg_test)
      y_score = y_prob[:, 1]  # Solo nos interesa la columna de la clase positiva

  # Calcula la curva ROC y el AUC para la clase positiva
  fpr, tpr, _ = roc_curve(y_true, y_score, pos_label=2)
  auc_LDA = auc(fpr, tpr)

  print(f"El AUC para la clase positiva es: {auc_LDA:.2f}")

else:

  # Binariza las etiquetas
  y_true_bin = label_binarize(labels_test, classes=np.unique(labels_test))

  # Asegúrate de que y_score es una matriz con una columna por clase
  # Reporte de clasificación
  y_true, y_pred = labels_test, best_lda.predict(eeg_test)
  print(classification_report(y_true, y_pred), end="\n\n")

  # Obtener la función de decisión o probabilidad para las clases
  try:
      y_score = best_lda.decision_function(eeg_test)
  except AttributeError:
      y_prob = best_lda.predict_proba(eeg_test)
      y_score = y_prob

  # Inicializa contenedores para FPR y TPR
  fpr = dict()
  tpr = dict()
  roc_auc = dict()

  # Calcula la curva ROC y el AUC para cada clase
  for i in range(y_true_bin.shape[1]):
      fpr[i], tpr[i], _ = roc_curve(y_true_bin[:, i], y_score[:, i])
      roc_auc[i] = auc(fpr[i], tpr[i])

  # Si deseas, puedes calcular una curva ROC y un AUC promedio para todas las clases
  fpr["macro"], tpr["macro"], _ = roc_curve(y_true_bin.ravel(), y_score.ravel())
  roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

  auc_LDA = roc_auc['macro']

  # Ahora puedes plotear las curvas ROC para cada clase o el promedio.
  print(f"El AUC promedio para todas las clases es: {auc_LDA:.2f}")

              precision    recall  f1-score   support

           0       0.40      0.67      0.50         6
           1       0.00      0.00      0.00         6

    accuracy                           0.33        12
   macro avg       0.20      0.33      0.25        12
weighted avg       0.20      0.33      0.25        12


El AUC para la clase positiva es: nan


c:\Users\Admin\miniconda3\envs\bcihack-GUI\Lib\site-packages\sklearn\metrics\_ranking.py:1132: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(


# SVM

In [8]:
# Instanciamos un SVM
svm = SVC(probability=True)  # instanciamos el clasificador SVM con la opción de probabilidad habilitada

### ********** Creamos el pipeline para SVM **********

pipeline_svm = Pipeline([
    ('pasabanda', filtro),
    ('cspmulticlase', csp),
    ('featureExtractor', featureExtractor),
    ('ravelTransformer', ravelTransformer),
    ('svm', svm)
])

### ********** Creamos la grilla de hiperparámetros **********

grid_svm = {
    'pasabanda__lowcut': [5],
    'pasabanda__highcut': [16],
    'cspmulticlase__n_components': [2, 3],
    'cspmulticlase__method': ["ovo","ova"],
    'cspmulticlase__reg': [0.01],
    'cspmulticlase__log': [None],
    'cspmulticlase__norm_trace': [False],
    'featureExtractor__method': ["welch", "hilbert"],
    'featureExtractor__sample_rate': [fm],
    'featureExtractor__band_values': [[8, 12]],
    'svm__C': [0.1, 1, 10],  # Parámetro de regularización
    'svm__kernel': ['linear', 'rbf'],  # Tipo de kernel
    'svm__gamma': ['scale', 'auto']  # Coeficiente de kernel para 'rbf'
}

# Creamos el RandomizedSearch para el SVM
# Aquí puedes ajustar n_iter según cuántas iteraciones de búsqueda aleatoria quieras realizar
random_search_svm = RandomizedSearchCV(pipeline_svm, param_distributions=grid_svm, n_iter=10, cv=5, n_jobs=1, verbose=0, random_state=42)

### ********** Entrenamos el modelo **********
random_search_svm.fit(eeg_train, labels_train)

### ******************************************

### Nos quedamos con el mejor estimador
best_svm = random_search_svm.best_estimator_

# Reporte de clasificación
y_true, y_pred = labels_test, best_svm.predict(eeg_test)
print(classification_report(y_true, y_pred), end="\n\n")

## Creamos una matriz de confusión
cm_svm = confusion_matrix(y_true, y_pred)
print(cm_svm)

## obtenemos precision, recall y f1-score y los guardamos en variables
precision_svm, recall_svm, f1score_svm, _ = precision_recall_fscore_support(y_true, y_pred, average='macro')

## Obtenemos el accuracy y lo redondeamos a 2 decimales
acc_svm = accuracy_score(y_true, y_pred)
acc_svm = np.round(acc_svm, decimals=2)*100
print(f"El accuracy del mejor clasificador SVM es de {acc_svm}")

Computing rank from data with rank=None
    Using tolerance 1.1 (2.2e-16 eps * 6 dim * 7.9e+14  max singular value)
    Estimated rank (mag): 6
    MAG: rank 6 computed from 6 data channels with 0 projectors
Reducing data rank from 6 -> 6
Estimating covariance using SHRINKAGE
Done.
Computing rank from data with rank=None
    Using tolerance 1.1 (2.2e-16 eps * 6 dim * 8.4e+14  max singular value)
    Estimated rank (mag): 6
    MAG: rank 6 computed from 6 data channels with 0 projectors
Reducing data rank from 6 -> 6
Estimating covariance using SHRINKAGE
Done.
Computing rank from data with rank=None
    Using tolerance 1.1 (2.2e-16 eps * 6 dim * 8.1e+14  max singular value)
    Estimated rank (mag): 6
    MAG: rank 6 computed from 6 data channels with 0 projectors
Reducing data rank from 6 -> 6
Estimating covariance using SHRINKAGE
Done.
Computing rank from data with rank=None
    Using tolerance 1.1 (2.2e-16 eps * 6 dim * 8.5e+14  max singular value)
    Estimated rank (mag): 6
    MAG

c:\Users\Admin\miniconda3\envs\bcihack-GUI\Lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
30 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
30 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\Admin\miniconda3\envs\bcihack-GUI\Lib\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\Admin\miniconda3\envs\bcihack-GUI\Lib\site-packages\sklearn\base.py", line 1151, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Admin\miniconda3\envs\bcihack-GUI\Lib\site-packa

In [9]:
if num_classes == 2:
  # y_prob = best_svm.predict_proba(eeg_test)

  # # Usa solo la columna correspondiente a la clase positiva para el score
  # y_score_pos_class = y_prob[:, 1]

  # # Calcula la curva ROC y el AUC para la clase positiva
  # fpr, tpr, _ = roc_curve(y_true_bin.ravel(), y_score_pos_class.ravel())
  # auc_SVM = auc(fpr, tpr)

  # print(f"El AUC para la clase positiva es: {auc_SVM:.2f}")
  y_prob = best_svm.predict_proba(eeg_test)

  # Define las etiquetas verdaderas en formato binario
  y_true_bin = np.where(y_true == 1, 1, 0)  # Asume que 1 es la clase positiva

  # Usa solo la columna correspondiente a la clase positiva para el score
  y_score_pos_class = y_prob[:, 1]

  # Calcula la curva ROC y el AUC para la clase positiva
  fpr, tpr, _ = roc_curve(y_true_bin.ravel(), y_score_pos_class.ravel())
  auc_SVM = auc(fpr, tpr)

  print(f"El AUC para la clase positiva es: {auc_SVM:.2f}")

else:
  y_prob = best_svm.predict_proba(eeg_test)

  # Calcular la curva ROC y el AUC
  n_classes = len(np.unique(labels_train))
  roc_aucs = []

  for i in range(n_classes):
      fpr, tpr, _ = roc_curve(labels_test == i, y_prob[:, i])

      if not np.any(np.isnan(tpr)):  # Solo calcula AUC si no hay NaNs
          roc_aucs.append(auc(fpr, tpr))

  auc_SVM = np.mean(roc_aucs)
  print(f"El AUC promedio del clasificador SVM es de {auc_SVM:.2f}")

El AUC para la clase positiva es: 0.39


# CRNN

In [10]:
from sklearn.base import TransformerMixin
from sklearn.preprocessing import StandardScaler
class ChannelScaler(TransformerMixin):
    def __init__(self):
        self.scalers = []

    def fit(self, X, y=None):
        n_channels = X.shape[1]
        self.scalers = [StandardScaler() for _ in range(n_channels)]
        for i in range(n_channels):
            self.scalers[i].fit(X[:, i, :])
        return self

    def transform(self, X, y=None):
        n_channels = X.shape[1]
        X_scaled = np.empty_like(X)
        for i in range(n_channels):
            X_scaled[:, i, :] = self.scalers[i].transform(X[:, i, :])
        return X_scaled

In [11]:
from CNNEEG import CRNN_EEGNet
# Filtramos los ensayos y etiquetas para mantener solo las clases deseadas
from keras.utils import to_categorical

pipeline = Pipeline([
    ('pasabanda', filtro),
    ('standardnormalization', ChannelScaler())])

# crnn_eegnet.fit(X_train, y_train, epochs=7000, validation_data=(X_val, y_val), patience=3000)
pipeline.fit(trials, labels)

transformed_trials = pipeline.transform(trials)
print(transformed_trials.shape)

labels_onehot = to_categorical(labels, num_classes = num_classes)

# # Dividir en conjunto de entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(transformed_trials, labels_onehot, test_size=0.3, random_state=42, stratify= labels_onehot)
X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size=0.5, random_state=42, stratify= y_val)

# X_train = np.expand_dims(X_train, axis=-1)
# X_val = np.expand_dims(X_val, axis=-1)
# X_test = np.expand_dims(X_test, axis=-1)

input_shape = X_train.shape[1:]  # Tomar la forma de los datos, excluyendo la dimensión de los ejemplos

crnn_eegnet = CRNN_EEGNet(input_shape= input_shape, num_classes = num_classes)
crnn_eegnet.compile()

fitness = crnn_eegnet.fit(X_train, y_train, epochs = 100, validation_data = (X_val, y_val), patience = 15)

(60, 6, 1000)


Epoch 1/100
2/2 [==============================] - ETA: 0s - loss: 0.7157 - accuracy: 0.4762
Epoch 1: val_loss improved from inf to 0.71862, saving model to best_model.h5
2/2 [==============================] - 6s 1s/step - loss: 0.7157 - accuracy: 0.4762 - val_loss: 0.7186 - val_accuracy: 0.1111
Epoch 2/100


c:\Users\Admin\miniconda3\envs\bcihack-GUI\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


2/2 [==============================] - ETA: 0s - loss: 0.6782 - accuracy: 0.5000
Epoch 2: val_loss did not improve from 0.71862
2/2 [==============================] - 1s 313ms/step - loss: 0.6782 - accuracy: 0.5000 - val_loss: 0.7203 - val_accuracy: 0.2222
Epoch 3/100
2/2 [==============================] - ETA: 0s - loss: 0.6130 - accuracy: 0.7381
Epoch 3: val_loss did not improve from 0.71862
2/2 [==============================] - 1s 300ms/step - loss: 0.6130 - accuracy: 0.7381 - val_loss: 0.7222 - val_accuracy: 0.2222
Epoch 4/100
2/2 [==============================] - ETA: 0s - loss: 0.6108 - accuracy: 0.7619
Epoch 4: val_loss did not improve from 0.71862
2/2 [==============================] - 1s 307ms/step - loss: 0.6108 - accuracy: 0.7619 - val_loss: 0.7240 - val_accuracy: 0.2222
Epoch 5/100
2/2 [==============================] - ETA: 0s - loss: 0.5796 - accuracy: 0.8095
Epoch 5: val_loss did not improve from 0.71862
2/2 [==============================] - 1s 304ms/step - loss: 0.57

In [12]:
# 1. Obtener las predicciones del modelo CRNN_EEGNet
probs = crnn_eegnet.model.predict(X_test)  # Esto nos da las probabilidades

y_pred = np.argmax(probs, axis=1)
y_true = np.argmax(y_test, axis=1)  # Convertir y_test de formato one-hot a índices de clases

print(classification_report(y_true, y_pred), end="\n\n")

cm_crnn = crnn_eegnet
# Matriz de confusión
cm_crnn = confusion_matrix(y_true, y_pred)
print(cm_crnn)

# Métricas individuales
precision_crnn, recall_crnn, f1score_crnn, _ = precision_recall_fscore_support(y_true, y_pred, average='macro')

# Accuracy
acc_crnn = accuracy_score(y_true, y_pred)
acc_crnn = np.round(acc_crnn, decimals=2) * 100
print(f"El accuracy del modelo CRNN_EEGNet es de {acc_crnn}")

# Calcular la curva ROC y el AUC
n_classes = len(np.unique(y_true))
roc_aucs = []

for i in range(n_classes):
    fpr, tpr, _ = roc_curve(y_true == i, probs[:, i])

    if not np.any(np.isnan(tpr)): # Solo calcula AUC si no hay NaNs
        roc_aucs.append(auc(fpr, tpr))

auc_CRNN = np.mean(roc_aucs)
print(f"El AUC promedio del modelo CRNN_EEGNet es de {auc_CRNN:.2f}")

1/1 [==============================] - 1s 577ms/step
              precision    recall  f1-score   support

           0       0.43      0.75      0.55         4
           1       0.50      0.20      0.29         5

    accuracy                           0.44         9
   macro avg       0.46      0.47      0.42         9
weighted avg       0.47      0.44      0.40         9


[[3 1]
 [4 1]]
El accuracy del modelo CRNN_EEGNet es de 44.0
El AUC promedio del modelo CRNN_EEGNet es de 0.30


# Entrenar clasificadores con todo el conjunto de datos

In [13]:
best_lda.fit(trials, labels)

Computing rank from data with rank=None
    Using tolerance 1.3 (2.2e-16 eps * 6 dim * 9.6e+14  max singular value)
    Estimated rank (mag): 6
    MAG: rank 6 computed from 6 data channels with 0 projectors
Reducing data rank from 6 -> 6
Estimating covariance using SHRINKAGE
Done.
Computing rank from data with rank=None
    Using tolerance 1.3 (2.2e-16 eps * 6 dim * 9.5e+14  max singular value)
    Estimated rank (mag): 6
    MAG: rank 6 computed from 6 data channels with 0 projectors
Reducing data rank from 6 -> 6
Estimating covariance using SHRINKAGE
Done.


Pipeline(steps=[('pasabanda', Filter(highcut=16, lowcut=5)),
                ('cspmulticlase', CSPMulticlass(n_classes=2, reg=0.01)),
                ('featureExtractor',
                 FeatureExtractor(band_values=[8, 12], method='hilbert')),
                ('ravelTransformer', RavelTransformer()),
                ('lda', LinearDiscriminantAnalysis(tol=0.01))])

In [14]:
best_svm.fit(trials, labels)

Computing rank from data with rank=None
    Using tolerance 1.3 (2.2e-16 eps * 6 dim * 9.6e+14  max singular value)
    Estimated rank (mag): 6
    MAG: rank 6 computed from 6 data channels with 0 projectors
Reducing data rank from 6 -> 6
Estimating covariance using SHRINKAGE
Done.
Computing rank from data with rank=None
    Using tolerance 1.3 (2.2e-16 eps * 6 dim * 9.5e+14  max singular value)
    Estimated rank (mag): 6
    MAG: rank 6 computed from 6 data channels with 0 projectors
Reducing data rank from 6 -> 6
Estimating covariance using SHRINKAGE
Done.


Pipeline(steps=[('pasabanda', Filter(highcut=16, lowcut=5)),
                ('cspmulticlase',
                 CSPMulticlass(n_classes=2, n_components=3, reg=0.01)),
                ('featureExtractor',
                 FeatureExtractor(band_values=[8, 12], method='hilbert')),
                ('ravelTransformer', RavelTransformer()),
                ('svm',
                 SVC(C=1, gamma='auto', kernel='linear', probability=True))])

# Creación de DataFrames

In [15]:
df = pd.DataFrame(columns=["Accuracy", "Precision", "Recall", "F1-Score", "AUC"])
df.loc["LDA"] = [acc_lda, precision_lda, recall_lda, f1score_lda, auc_LDA]
df.loc["SVM"] = [acc_svm, precision_svm, recall_svm, f1score_svm, auc_SVM]
df.loc["CRNN"] = [acc_crnn,precision_crnn, recall_crnn, f1score_crnn, auc_CRNN]  # We only have accuracy for CRNN in this case

print(df)

# Save and return the best estimator for each pipeline
best_estimators = {}
best_estimators['LDA'] = best_lda
best_estimators['SVM'] = best_svm
best_estimators['CRNN'] = crnn_eegnet

# Save and return a DataFrame with the best hyperparameters for each pipeline
hyperparameters_data = {'LDA': [random_search_lda.best_params_], 'SVM': [random_search_svm.best_params_]}

hyperparameters_df = pd.DataFrame(hyperparameters_data, index=['Best Parameters'])

#Matrices de confusión

cm = {'LDA': cm_lda,
      'SVM': cm_svm,
      'CRNN': cm_crnn}

      Accuracy  Precision    Recall  F1-Score       AUC
LDA       33.0   0.200000  0.333333  0.250000       NaN
SVM       42.0   0.414286  0.416667  0.412587  0.388889
CRNN      44.0   0.464286  0.475000  0.415584  0.300000


In [16]:
# DataFrame with accuracy and other metrics
results_df_path = "/content/results_df.csv"
df.to_csv(results_df_path)

# DataFrame with best hyperparameters
hyperparameters_df_path = "/content/hyperparameters_df.csv"
hyperparameters_df.to_csv(hyperparameters_df_path)

# Save best estimators
best_estimators_paths = {
    'LDA': "/content/best_lda.pkl",
    'SVM': "/content/best_svm.pkl",
    'CRNN': "/content/best_crnn.pkl"
}

for model_name, path in best_estimators_paths.items():
    with open(path, 'wb') as file:
        pickle.dump(best_estimators[model_name], file)

#Guardando matrices de confusión
with open("/content/cm.pkl", 'wb') as file:
        pickle.dump(cm, file)

OSError: Cannot save file into a non-existent directory: '\content'